In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
import sys
sys.path.append('/home/anton/dima_experiments/my_dissertation')
from input_creator.scripts.readers import ais_reader, file_by_district

In [22]:
len(glob.glob('/home/anton/dima_experiments/my_dissertation/conceptual_runs/cal_res/gr4j/*.csv'))

537

In [5]:
russia_ws = gpd.read_file('/home/anton/dima_experiments/my_dissertation/geo_data/great_db/geometry/russia_ws.gpkg')
russia_ws = russia_ws.set_index('gauge_id')
russia_ws

,name_ru,name_en,new_area,ais_dif,geometry
gauge_id,,,,,
9002,р.Енисей - г.Кызыл,r.Enisej - g.Kyzyl,115263.989,0.230,"POLYGON ((96.87792 53.72792, 96.87792 53.72708..."
9022,р.Енисей - пос.Никитино,r.Enisej - pos.Nikitino,184499.118,1.373,"POLYGON ((96.87792 53.72708, 96.88042 53.72708..."
9053,р.Енисей - пос.Базаиха,r.Enisej - pos.Bazaiha,302690.417,0.897,"POLYGON ((92.38292 56.11042, 92.38292 56.10958..."
9112,р.Большой Енисей - с.Тоора-Хем,r.Bol'shoj Enisej - s.Toora-Hem,15448.834,0.969,"POLYGON ((97.07125 52.71625, 97.07125 52.71542..."
9115,р.Большой Енисей - пос.Сейба,r.Bol'shoj Enisej - pos.Sejba,44934.891,0.751,"POLYGON ((96.87792 53.72708, 96.88042 53.72708..."
...,...,...,...,...,...
11056,р.Иртыш - г.Тобольск,r.Irtysh - g.Tobol'sk,1540451.386,2.697,"POLYGON ((65.28125 59.37208, 65.28125 59.37125..."
11061,р.Иртыш - г.Ханты-Мансийск,r.Irtysh - g.Hanty-Mansijsk,1694019.131,2.668,"POLYGON ((62.66375 61.80542, 62.66542 61.80542..."
11801,р.Обь - г.Салехард,r.Ob' - g.Salehard,3004250.818,1.839,"POLYGON ((66.69042 66.42875, 66.69042 66.42792..."


# Parsing

In [6]:
height_bs = pd.read_csv('./data/height_gauge.csv',
                        index_col='gauge_id')
height_bs

,name,height
gauge_id,,
7003,Голоустная - р.Голоустная - с.Большое Голоустное,466.03
7004,Бугульдейка - р.Бугульдейка - д.Большая Бугуль...,468.50
7006,Анга - р.Анга - с.Еланцы,465.73
7008,Рель - р.Рель - с.Байкальское,477.14
7009,ТЫЯ - р.Тыя - гп Тыя,504.99
...,...,...
1103,без названия - руч.Морозова - Водопадный,1166.74
5472,Мули - р.Мули - пос.Высокогорный,448.88
6059,Газимур - р.Газимур - Газимурский Завод,677.74


### From by year to time series 

#### Level

In [6]:
file_by_district(all_files=glob.glob('./data/res_levels/*/*.csv'),
                 height_bs=height_bs,
                 save_storage='../geo_data/great_db/ais_data',
                 variable_tag='h')

#### Discharge

In [7]:
file_by_district(all_files=glob.glob('./data/res_discharge/*/*.csv'),
                 height_bs=height_bs,
                 save_storage='../geo_data/great_db/ais_data',
                 variable_tag='q')

#### All together

In [8]:
level_storage = '../geo_data/great_db/ais_data/h'
discharge_storage = '../geo_data/great_db/ais_data/q'
res_storage = '../geo_data/great_db/ais_data/q_h'

In [17]:
len(glob.glob('../geo_data/great_db/ais_data/q_h/*.csv'))

2814

In [18]:
for gauge in height_bs.index:
    lvl_file = pd.read_csv(f'{level_storage}/{gauge}.csv',
                            index_col='date')
    lvl_file.index = pd.to_datetime(lvl_file.index)
    lvl_bs = height_bs.loc[gauge, 'height']
    lvl_file['lvl_mbs'] = lvl_bs + \
        lvl_file['level'] * 1e-2  # type: ignore
    lvl_file = lvl_file.rename(columns={'level': 'lvl_sm'})
    try:
        dis_file = pd.read_csv(f'{discharge_storage}/{gauge}.csv',
                               index_col='date')
        dis_file.index = pd.to_datetime(dis_file.index)
        try:
            ws_area = russia_ws.loc[f'{gauge}', 'new_area']
        except KeyError:
            print(f'Для поста {gauge} нет площади ):')
            ws_area = np.NaN
        dis_file['q_mm_day'] = (dis_file['discharge'] * 86400)/(ws_area * 1e3)
        dis_file = dis_file.rename(columns={'discharge': 'q_cms_s'})
        res_file = pd.concat([dis_file, lvl_file], axis=1)
        res_file = res_file[['lvl_sm', 'q_cms_s', 'lvl_mbs', 'q_mm_day']]
        
    except Exception:
        dis_file = pd.DataFrame()
        dis_file.index = pd.date_range(start='01/01/2008',
                                       end='12/31/2020')
        dis_file['date'] = pd.date_range(start='01/01/2008',
                                         end='12/31/2020')
        dis_file = dis_file.set_index('date')
        dis_file['q_cms_s'] = np.NaN
        dis_file['q_mm_day'] = np.NaN
        res_file = pd.concat([dis_file, lvl_file], axis=1)
        res_file = res_file[['lvl_sm', 'q_cms_s', 'lvl_mbs', 'q_mm_day']]

    res_file.to_csv(f'{res_storage}/{gauge}.csv')

        

Для поста 2253 нет площади ):
Для поста 83333 нет площади ):
Для поста 83335 нет площади ):
Для поста 83801 нет площади ):
Для поста 83809 нет площади ):
Для поста 83813 нет площади ):
Для поста 83818 нет площади ):
Для поста 83829 нет площади ):
Для поста 83833 нет площади ):
Для поста 81562 нет площади ):
Для поста 81565 нет площади ):
Для поста 81580 нет площади ):
Для поста 81581 нет площади ):
Для поста 81592 нет площади ):
Для поста 81593 нет площади ):
Для поста 81597 нет площади ):
Для поста 81600 нет площади ):
Для поста 81602 нет площади ):
Для поста 81604 нет площади ):
Для поста 81616 нет площади ):
Для поста 81640 нет площади ):
Для поста 81648 нет площади ):
Для поста 81674 нет площади ):
Для поста 81677 нет площади ):
Для поста 81686 нет площади ):
Для поста 81691 нет площади ):
Для поста 81692 нет площади ):
Для поста 81693 нет площади ):
Для поста 81694 нет площади ):
Для поста 81697 нет площади ):
Для поста 81717 нет площади ):
Для поста 81724 нет площади ):
Для поста